# Предсказываем стоимость мед страховки



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df = pd.read_csv('insurance.csv')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную

In [ ]:
X = df[['age',	'sex',	'bmi',	'children',	'smoker',	'region_northeast', 'region_northwest',	'region_southeast',	'region_southwest']]
y = df['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

lr = LinearRegression() 
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание

lr.coef_

array([  258.14709548,   -36.44901107,   356.95652853,   370.4462921 ,
       24141.79291538,   565.7955346 ,   161.31018894,  -564.48753426,
        -162.61818928])

In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [ ]:
lr.predict(X_test)

array([ 4.67871804e+03,  3.07990527e+04,  9.42722712e+03,  9.82305051e+03,
        1.57402032e+04,  7.10290758e+03,  2.60192484e+03,  1.51513020e+04,
        1.21854318e+04,  1.08162544e+04,  1.16616372e+04,  1.39592355e+04,
        1.37359756e+04,  1.35635450e+03,  1.16538607e+04,  1.12937978e+04,
        1.00853970e+04,  3.45102954e+04,  2.88450214e+04,  8.90041975e+03,
        3.35835290e+04, -2.01559192e+03,  1.20317900e+04,  3.82755964e+03,
        8.30841133e+03,  2.74539065e+04,  3.26642506e+03,  6.44134455e+03,
        8.46796110e+03,  4.08632782e+03,  1.24786606e+04,  8.57222064e+03,
        2.76938208e+04,  1.27976483e+04,  3.31370648e+04,  4.24502537e+03,
       -7.52039896e+02,  1.07328214e+04,  1.14898297e+04,  1.39632151e+04,
        1.07271262e+04,  1.03692425e+04,  4.65275030e+03,  1.27114692e+04,
        2.72343309e+04,  9.71571613e+03,  1.24439411e+04,  7.15186894e+03,
        9.98706924e+03,  4.63486134e+03,  2.99677710e+02,  3.37445166e+03,
        1.05017188e+04, -

In [ ]:
mean_squared_error(y_test, lr.predict(X_test)) #на тестовой выборке

42023926.00916992

In [ ]:
mean_squared_error(y_train, lr.predict(X_train)) #на обучающей выборке

35189933.37645373

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([27041.79196478])